# All experiments using our approach for various missingness rates and types

In [2]:
import simulate
import baseline
import dataload
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#import pypots
from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import BRITS
from pypots.utils.metrics import cal_mae, cal_rmse, cal_mre
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#import pypots
from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import BRITS
import torch
from pypots.utils.metrics import cal_mae
from pypots.utils.metrics import cal_binary_classification_metrics
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
import seaborn as sns
import numpy as np 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from sklearn import metrics
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
import os

In [3]:
dict = dataload.uci()
X = dict["X"]
y = dict["y"]

File Path :  C:/Users/Ronak/Desktop/Data_imputation/PyPOTS-main/ece209as_project-main/ece209as_project-main/data/UCI_HAR_Dataset/train/Inertial_Signals/
File Path :  C:/Users/Ronak/Desktop/Data_imputation/PyPOTS-main/ece209as_project-main/ece209as_project-main/data/UCI_HAR_Dataset/test/Inertial_Signals/


In [4]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.1, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

# MAR - logistic
dict_MAR = simulate.simulate_nan(X, 0.1, "MAR")
X_intact_MAR = dict_MAR['X_init']
X_missing_MAR = dict_MAR['X_incomp']
X_mask_MAR = dict_MAR['mask']

# MNAR - logistic
dict_MAR = simulate.simulate_nan(X, 0.1, "MNAR", opt="logistic")
X_intact_MNAR = dict_MAR['X_init']
X_missing_MNAR = dict_MAR['X_incomp']
X_mask_MNAR = dict_MAR['mask']

X_intact = [X_intact_MCAR, X_intact_MAR, X_intact_MNAR]
X_missing = [X_missing_MCAR, X_missing_MAR, X_missing_MNAR]
X_mask = [X_mask_MCAR, X_mask_MAR, X_mask_MNAR]
missingness = ["MCAR", "MAR", "MNAR"]

In [20]:
result_uci_imputation = []

In [ ]:
res = []
imputated_data = []

for i in range(1):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, batch_size=256, epochs=100,device=torch.device('cuda:0'))
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    imputated_data.append(imputation)
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))
    

result_uci_imputation.append(res)

In [46]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness:
    for j in errors:
        cols.append(i+" "+j)

methods = ["BRITS"]
result_uci_imputation_df = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)

In [47]:
result_uci_imputation_df

,MCAR mae,MCAR rmse,MCAR mre,MAR mae,MAR rmse,MAR mre,MNAR mae,MNAR rmse,MNAR mre
BRITS,0.123852,0.22107,0.511,0.128857,0.225485,0.534,0.123321,0.217309,0.509


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[0],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [9]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [10]:
checkpoint = ModelCheckpoint("har_weights.h5", monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [11]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


Epoch 1/50
365/365 [==============================] - 5s 12ms/step - loss: 0.3731 - accuracy: 0.8553 - val_loss: 0.1764 - val_accuracy: 0.9240
Epoch 2/50
365/365 [==============================] - 5s 13ms/step - loss: 0.1505 - accuracy: 0.9388 - val_loss: 0.1432 - val_accuracy: 0.9553
Epoch 3/50
365/365 [==============================] - 6s 17ms/step - loss: 0.1310 - accuracy: 0.9459 - val_loss: 0.1485 - val_accuracy: 0.9313
Epoch 4/50
365/365 [==============================] - 4s 12ms/step - loss: 0.1231 - accuracy: 0.9485 - val_loss: 0.1117 - val_accuracy: 0.9473
Epoch 5/50
365/365 [==============================] - 4s 12ms/step - loss: 0.1178 - accuracy: 0.9510 - val_loss: 0.1063 - val_accuracy: 0.9547
Epoch 6/50
365/365 [==============================] - 4s 12ms/step - loss: 0.1112 - accuracy: 0.9538 - val_loss: 0.1436 - val_accuracy: 0.9500
Epoch 7/50
365/365 [==============================] - 4s 12ms/step - loss: 0.1229 - accuracy: 0.9499 - val_loss: 0.1012 - val_accuracy: 0.9600

In [38]:

imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.0281 - accuracy: 0.9465
HAR - CNN Test Accuracy: 91.17%


# Classification performance on MAR type missingness

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[1],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [ ]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [ ]:
checkpoint = ModelCheckpoint("har_weights.h5", monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


In [37]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.0381 - accuracy: 0.8901
HAR - CNN Test Accuracy: 86.62%


# Classification performance on MNAR type missingness

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[2],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [ ]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [ ]:
checkpoint = ModelCheckpoint("har_weights.h5", monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


In [36]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.0321 - accuracy: 0.9111
HAR - CNN Test Accuracy: 89.85%


# UCI HAR BRITS with Missingness at 20% 

In [15]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.2, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

X_intact = [X_intact_MCAR]
X_missing = [X_missing_MCAR]
X_mask = [X_mask_MCAR]
missingness = ["MCAR"]

In [16]:
result_uci_imputation = []

In [ ]:
res = []
imputated_data = []

for i in range(1):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, batch_size=256, epochs=100,device=torch.device('cuda:0'))
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    imputated_data.append(imputation)
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

In [51]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness[:1]:
    for j in errors:
        cols.append(i+" "+j)

methods = ["BRITS"]
result_uci_imputation_df = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)
result_uci_imputation_df

,MCAR mae,MCAR rmse,MCAR mre
BRITS,0.103246,0.184325,0.957


In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[0],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [21]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [22]:
checkpoint = ModelCheckpoint("har_weights.h5", monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [23]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


Epoch 1/50
365/365 [==============================] - 4s 11ms/step - loss: 0.3869 - accuracy: 0.8456 - val_loss: 0.1847 - val_accuracy: 0.9360
Epoch 2/50
365/365 [==============================] - 4s 11ms/step - loss: 0.1575 - accuracy: 0.9388 - val_loss: 0.1424 - val_accuracy: 0.9493
Epoch 3/50
365/365 [==============================] - 4s 11ms/step - loss: 0.1286 - accuracy: 0.9481 - val_loss: 0.1268 - val_accuracy: 0.9560
Epoch 4/50
365/365 [==============================] - 4s 11ms/step - loss: 0.1202 - accuracy: 0.9505 - val_loss: 0.1343 - val_accuracy: 0.9507
Epoch 5/50
365/365 [==============================] - 4s 11ms/step - loss: 0.1125 - accuracy: 0.9559 - val_loss: 0.1103 - val_accuracy: 0.9573
Epoch 6/50
365/365 [==============================] - 4s 10ms/step - loss: 0.1163 - accuracy: 0.9542 - val_loss: 0.2458 - val_accuracy: 0.9173
Epoch 7/50
365/365 [==============================] - 4s 11ms/step - loss: 0.1301 - accuracy: 0.9477 - val_loss: 0.1171 - val_accuracy: 0.9453

In [35]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.0311 - accuracy: 0.9285
HAR - CNN Test Accuracy: 90.73%


# UCI HAR BRITS with Missingness at 30% 

In [25]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.3, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

X_intact = [X_intact_MCAR]
X_missing = [X_missing_MCAR]
X_mask = [X_mask_MCAR]
missingness = ["MCAR"]

In [26]:
result_uci_imputation = []

In [ ]:
res = []
imputated_data = []

for i in range(1):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, batch_size=256, epochs=100,device=torch.device('cuda:0'))
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    imputated_data.append(imputation)
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

In [54]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness[:1]:
    for j in errors:
        cols.append(i+" "+j)

methods = ["BRITS"]
result_uci_imputation_df = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)
result_uci_imputation_df

,MCAR mae,MCAR rmse,MCAR mre
BRITS,0.095643,0.186893,0.997


In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[0],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [30]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [32]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


Epoch 1/50
365/365 [==============================] - 5s 12ms/step - loss: 0.3758 - accuracy: 0.8493 - val_loss: 0.1955 - val_accuracy: 0.9220
Epoch 2/50
365/365 [==============================] - 4s 11ms/step - loss: 0.1583 - accuracy: 0.9346 - val_loss: 0.1402 - val_accuracy: 0.9420
Epoch 3/50
365/365 [==============================] - 4s 11ms/step - loss: 0.1245 - accuracy: 0.9475 - val_loss: 0.1221 - val_accuracy: 0.9580
Epoch 4/50
365/365 [==============================] - 5s 13ms/step - loss: 0.1150 - accuracy: 0.9525 - val_loss: 0.1313 - val_accuracy: 0.9513
Epoch 5/50
365/365 [==============================] - 4s 12ms/step - loss: 0.1148 - accuracy: 0.9529 - val_loss: 0.1224 - val_accuracy: 0.9527
Epoch 6/50
365/365 [==============================] - 4s 11ms/step - loss: 0.1159 - accuracy: 0.9519 - val_loss: 0.1212 - val_accuracy: 0.9467
Epoch 7/50
365/365 [==============================] - 5s 12ms/step - loss: 0.1058 - accuracy: 0.9566 - val_loss: 0.1116 - val_accuracy: 0.9620

In [33]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.0411 - accuracy: 0.8455
HAR - CNN Test Accuracy: 82.62%


# UCI HAR BRITS with Missingness at 40% 

In [34]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.4, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

X_intact = [X_intact_MCAR]
X_missing = [X_missing_MCAR]
X_mask = [X_mask_MCAR]
missingness = ["MCAR"]

In [35]:
result_uci_imputation = []

In [ ]:
res = []
imputated_data = []

for i in range(1):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, batch_size=256, epochs=100,device=torch.device('cuda:0'))
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    imputated_data.append(imputation)
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

In [59]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness[:1]:
    for j in errors:
        cols.append(i+" "+j)

methods = ["BRITS"]
result_uci_imputation_df = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)
result_uci_imputation_df

,MCAR mae,MCAR rmse,MCAR mre
BRITS,0.113654,0.189329,0.999


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[0],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [ ]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


In [30]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.0602 - accuracy: 0.8121
HAR - CNN Test Accuracy: 76.86%


# UCI HAR BRITS with Missingness at 50% 

In [37]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.5, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

X_intact = [X_intact_MCAR]
X_missing = [X_missing_MCAR]
X_mask = [X_mask_MCAR]
missingness = ["MCAR"]

In [38]:
result_uci_imputation = []

In [ ]:

res = []
imputated_data = []

for i in range(1):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, batch_size=256, epochs=100,device=torch.device('cuda:0'))
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    imputated_data.append(imputation)
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

In [62]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness[:1]:
    for j in errors:
        cols.append(i+" "+j)

methods = ["BRITS"]
result_uci_imputation_df = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)
result_uci_imputation_df

,MCAR mae,MCAR rmse,MCAR mre
BRITS,0.113654,0.19139,1.1


In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[0],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [42]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


In [29]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.112 - accuracy: 0.7001
HAR - CNN Test Accuracy: 68.43%


# UCI HAR BRITS with Missingness at 60% 

In [45]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.6, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

X_intact = [X_intact_MCAR]
X_missing = [X_missing_MCAR]
X_mask = [X_mask_MCAR]
missingness = ["MCAR"]

In [46]:
result_uci_imputation = []

In [ ]:
res = []
imputated_data = []

for i in range(1):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, batch_size=256, epochs=100,device=torch.device('cuda:0'))
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    imputated_data.append(imputation)
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

In [64]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness[:1]:
    for j in errors:
        cols.append(i+" "+j)

methods = ["BRITS"]
result_uci_imputation_df = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)
result_uci_imputation_df

,MCAR mae,MCAR rmse,MCAR mre
BRITS,0.115624,0.198277,11.3


In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[0],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [50]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


In [28]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.112 - accuracy: 0.6422
HAR - CNN Test Accuracy: 59.14%


# UCI HAR BRITS with Missingness at 70% 

In [53]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.7, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

X_intact = [X_intact_MCAR]
X_missing = [X_missing_MCAR]
X_mask = [X_mask_MCAR]
missingness = ["MCAR"]

In [ ]:
result_uci_imputation = []

In [ ]:
res = []
imputated_data = []

for i in range(1):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, batch_size=256, epochs=100,device=torch.device('cuda:0'))
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    imputated_data.append(imputation)
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

In [67]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness[:1]:
    for j in errors:
        cols.append(i+" "+j)

methods = ["BRITS"]
result_uci_imputation_df = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)
result_uci_imputation_df

,MCAR mae,MCAR rmse,MCAR mre
BRITS,0.115988,0.199835,1.23


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[0],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [ ]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


In [26]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.1912 - accuracy: 0.4617
HAR - CNN Test Accuracy: 48.22%


# UCI HAR BRITS with Missingness at 80% 

In [ ]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.8, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

X_intact = [X_intact_MCAR]
X_missing = [X_missing_MCAR]
X_mask = [X_mask_MCAR]
missingness = ["MCAR"]

In [ ]:
result_uci_imputation = []

In [ ]:
res = []
imputated_data = []

for i in range(1):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, batch_size=256, epochs=100,device=torch.device('cuda:0'))
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    imputated_data.append(imputation)
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

In [69]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness[:1]:
    for j in errors:
        cols.append(i+" "+j)

methods = ["BRITS"]
result_uci_imputation_df = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)
result_uci_imputation_df

,MCAR mae,MCAR rmse,MCAR mre
BRITS,0.118903,0.21341,1.83


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[0],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [ ]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


In [25]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.1912 - accuracy: 0.4617
HAR - CNN Test Accuracy: 43.86%


# UCI HAR BRITS with Missingness at 90% 

In [ ]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.8, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

X_intact = [X_intact_MCAR]
X_missing = [X_missing_MCAR]
X_mask = [X_mask_MCAR]
missingness = ["MCAR"]

In [ ]:
result_uci_imputation = []

In [ ]:
res = []
imputated_data = []

for i in range(1):
    brits = BRITS(n_steps=128, n_features=9,rnn_hidden_size=64, batch_size=256, epochs=100,device=torch.device('cuda:0'))
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    imputated_data.append(imputation)
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_uci_imputation.append(res)

In [71]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness[:1]:
    for j in errors:
        cols.append(i+" "+j)

methods = ["BRITS"]
result_uci_imputation_df = pd.DataFrame(result_uci_imputation, columns = cols, index=methods)
result_uci_imputation_df

,MCAR mae,MCAR rmse,MCAR mre
BRITS,0.123216,0.23548,2.13


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_intact[0],y,test_size=1500/len(y),random_state=42,stratify=y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=1500/len(Y_train),random_state=42,stratify=Y_train)
# zero-offset class values
Y_train = Y_train - 1
Y_test = Y_test - 1
Y_val = Y_val - 1
def to_categorical(y):
    num_classes = len(np.unique(y))
    return np.eye(num_classes, dtype='uint8')[y]
# one hot encode y
Y_train = np.squeeze(to_categorical(Y_train),axis=1)
Y_test = np.squeeze(to_categorical(Y_test),axis=1)
Y_val = np.squeeze(to_categorical(Y_val),axis=1)


print('X_train.shape : ', X_train.shape)
print('Y_train.shape : ', Y_train.shape)
print('X_test.shape : ', X_test.shape)
print('Y_test.shape : ', Y_test.shape)
print('X_test.shape : ', X_val.shape)
print('Y_test.shape : ', Y_val.shape)

X_train.shape :  (7299, 128, 9)
Y_train.shape :  (7299, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)
X_test.shape :  (1500, 128, 9)
Y_test.shape :  (1500, 6)


In [ ]:
verbose = 1
epochs = 50
batch_size = 20

n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_outputs = Y_train.shape[1]

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding = 'same'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                    epochs=epochs, batch_size=batch_size, verbose=verbose)


# evaluate model
(loss, accuracy) = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=verbose)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


In [24]:
imp_y = np.squeeze(to_categorical(y-1),axis=1)
scores = model.evaluate(imputated_data[0], imp_y, batch_size = batch_size, verbose = verbose)
print("HAR - CNN Test Accuracy: %.2f%%" % (scores[1]*100))

515/515 [==============================] - 2s 3ms/step - loss: 0.1912 - accuracy: 0.4617
HAR - CNN Test Accuracy: 39.45%
